In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from typing import Dict, List, Tuple, Union, Any

import cvxpy as cp
import numpy as np
import pandas as pd

from src.operators import negation
from src.misc import process_neg, Predicate, timer

# from src.preprocess_fol_dual import FOLConverter

# from src.constraints import ConstraintsConstructor
# from src.objective_function import specimen_construct_objective_function

# symbols_1 = ['¬', '∧', '∨', '⊗', '⊕', '→']
# symbols_2 = ['∀', '∃']
# symbols_3 = ['+', '-']
# symbols = symbols_1 + symbols_2 + symbols_3

In [3]:
data_dir_path = "./inputs/toy_data_loss" # たまたまファイル名が都合よいだけで，loss 関数は関係がない

file_names_dict = {
    'supervised': ['L_p1.csv', 'L_p2.csv', 'L_p3.csv'],
    'unsupervised': ['U.csv'],
    'rule': ['rules.txt']
}

In [4]:
import os
from typing import List, Dict, Tuple, Union
import sympy as sp
import pandas as pd
import numpy as np
from src.misc import is_symbol

# from .setup_problem import Setup
class Setup_:
    """
    型ヒント用（circular import の回避のため）
    """
    # def __init__(self):
    #     pass
    
    # FOLConverter の簡易動作確認用
    def __init__(self):
        self.len_j = 3
        self.len_u = 6


class FOLConverter:

    def __init__(self, obj: Setup_, file_path: str) -> None:
        self.obj = obj

        self.file_path = file_path
        self.KB_origin = self._construct_KB()
        self.KB = None

        self.KB_info = None

        self.M = None
        self.q = None

        self.predicates_dict_tmp = None
        
    def _construct_KB(self) -> List[List[str]]:
        """
        KB の .txt もしくは .tsv ファイルを読み込む
        """    
        KB_origin = []
        
        if "tsv" in self.file_path:
            KB_info = pd.read_table(self.file_path)
            rules = KB_info['formula']

            for line in rules:
                formula = line.split()
                KB_origin.append(formula)
        else:
            with open(self.file_path, 'r') as file:
                for line in file:
                    formula = line.split()
                    KB_origin.append(formula)

        return KB_origin

    def _identify_predicates(self, KB: List[List[str]]) -> Dict[str, sp.Symbol]:
        """
        KB 内の述語を特定し，
        各述語の係数を取り出すために
        sympy.Symbol で表現する
        """
        predicates = []

        for formula in KB:
            for item in formula:
                # if item not in ['¬', '∧', '∨', '⊗', '⊕', '→'] and item not in predicates:
                if not is_symbol(item) and item not in predicates:
                    predicates.append(item)
        
        predicates_dict = {predicate: sp.Symbol(predicate) for predicate in predicates}

        return predicates_dict

    # def _check_implication(self, formula: List[Union[str, sp.Symbol]]):
    def _check_implication(self, formula: List[str]) -> Tuple[bool, Union[None, int]]:
        """
        formula (リスト) について，
        含意記号 '→' の数を調べ，
        その数が 1 以下になっているかを確認する
        """
        
        # 実質，ここには 1 つのインデックスしか入らない
        implication_idxs = []

        for i, item in enumerate(formula):
            if item == '→':
                implication_idxs.append(i)
        
        implication_num = len(implication_idxs)
        
        if implication_num == 0:
            return False, None
        elif implication_num == 1:
            implication_idx = implication_idxs[0]
            return True, implication_idx
        else:
            print('this formula may be invalid')

    def _eliminate_implication(self, formula: List[str]) -> List[str]:
        """
        formula (リスト) 内に含意記号 '→' あれば変換し，消去する 
        """
        implication_flag, target_idx = self._check_implication(formula)

        if implication_flag:
            # 含意記号 '→' を境に formula (list) を 2 つに分ける
            x = formula[:target_idx]
            y = formula[target_idx + 1:]

            # x → y = ¬ x ⊕ y
            x_new = negation(x)
            y_new = y
            new_operator = ['⊕']

            new_formula = x_new + new_operator + y_new
        else:
            new_formula = formula

        return new_formula
       
    def _get_neg_idx_list(self, formula: List[str]) -> Tuple[List[str], List[str]]:
        """
        formula (リスト) 内の '¬' のインデックスリストと
        '¬' 以外のインデックスリストを返す
        """
        neg_idxs = []
        not_neg_idxs = []

        for i, item in enumerate(formula):
            if item == '¬':
                neg_idxs.append(i)
            else:
                not_neg_idxs.append(i)
        
        return neg_idxs, not_neg_idxs

    def _split_neg_idx_list(self, idx_list) -> List[List[int]]:
        """
        インデックスリストを連続する部分リストに分割する
        """
        result = []
        tmp = []

        for i in range(len(idx_list)):
            if not tmp or idx_list[i] == tmp[-1] + 1:
                tmp.append(idx_list[i])
            else:
                result.append(tmp)
                tmp = [idx_list[i]]

        if tmp:
            result.append(tmp)

        return result
    
    def _eliminate_multi_negations(self, formula: List[str]) -> List[str]:
        """
        formula (リスト) 内の連続する '¬' を削除する
        """
        neg_idxs, not_neg_idxs = self._get_neg_idx_list(formula)
        neg_idxs_decomposed = self._split_neg_idx_list(neg_idxs)

        neg_idxs_new = []
        for tmp in neg_idxs_decomposed:
            if len(tmp) % 2 == 0:
                pass
            else:
                neg_idxs_new.append(tmp[0])
        
        idxs_new = sorted(neg_idxs_new + not_neg_idxs)
        
        formula_new = []
        for idx in idxs_new:
            item = formula[idx]
            formula_new.append(item)

        return formula_new
    
    def convert_KB_origin(self) -> None:
        self.KB = []
        for formula in self.KB_origin:
            new_formula = self._eliminate_multi_negations(formula)
            new_formula = self._eliminate_implication(new_formula)
            self.KB.append(new_formula)
    
    def calculate_M_and_q(self) -> None:
        self.predicates_dict_tmp = self._identify_predicates(self.KB)

        # sympy で predicate を構成した KB
        # （formula を式変形した後の predicate の係数を取り出すため）
        KB_tmp = []
        for formula in self.KB:

            tmp_formula = []
            for item in formula:
                if item in self.predicates_dict_tmp.keys():
                    tmp_formula.append(self.predicates_dict_tmp[item])
                else:
                    tmp_formula.append(item)
                
            process_neg(tmp_formula)

            phi_h = []
            new_formula_1 = [sp.Symbol('1')]
            new_formula_2 = []

            tmp_new_formula_2 = 0
            for item in tmp_formula:
                if not is_symbol(item):
                    tmp_new_formula_2 += item
            
            new_formula_2.append(tmp_new_formula_2)

            phi_h.append(new_formula_1)
            phi_h.append(new_formula_2)
        
            KB_tmp.append(phi_h)

        predicates = list(self.predicates_dict_tmp.values())

        self.M = []
        self.q = []

        for phi_h in KB_tmp:

            base_M_h = np.zeros((len(phi_h), self.obj.len_j))
            base_q_h = np.zeros((len(phi_h), 1))
            for i, formula in enumerate(phi_h):
                for j, predicate in enumerate(predicates):

                    # negation
                    val = sp.Symbol('1') - formula[0]
                    coefficient = val.coeff(predicate)
                    base_M_h[i, j] = coefficient
                
                # negation
                val = sp.Symbol('1') - formula[0]
                base_q_h[i] = val.coeff(sp.Symbol('1'))

            tmp_M_h = []
            for i in range(self.obj.len_j):
                column = base_M_h[:, i]
                zeros = np.zeros((len(phi_h), self.obj.len_u - 1))
                concatenated_column = np.concatenate((column[:, np.newaxis], zeros), axis=1)
                tmp_M_h.append(concatenated_column)

            tmp_M_h = [np.concatenate(tmp_M_h, axis=1)]
            
            shifted_M_h = tmp_M_h[0]

            for i in range(self.obj.len_u - 1):
                shifted_M_h = np.roll(shifted_M_h, 1, axis=1)
                tmp_M_h.append(shifted_M_h)
            
            M_h = np.concatenate(tmp_M_h, axis=0)
            self.M.append(M_h)

            
            tmp_q_h = [base_q_h for u in range(self.obj.len_u)]
            q_h = np.concatenate(tmp_q_h, axis=0)
            self.q.append(q_h)
        
        # self.M = np.array(tmp_M)
        # self.q = np.array(tmp_q)

    def main(self) -> Tuple[Union[None, pd.DataFrame], List[List[str]], List[List[str]], List[np.ndarray], List[np.ndarray], Dict[str, sp.Symbol]]:
        """
        KB,
        KB_tmp,
        predicates_dict

        をそれぞれ計算する
        """
        self.convert_KB_origin()
        self.calculate_M_and_q()

        return self.KB_info, self.KB_origin, self.KB, self.M, self.q, self.predicates_dict_tmp
        
        

In [77]:

class Setup:
    """
    cvxpy.Problem に渡す objective function と constraints
    の生成

    インスタンス化の際に以下の 2 つを引数として渡す
    
    data_dir_path = "./inputs/toy_data"

    file_names_dict = {
        'supervised': ['L1', 'L2', 'L3'],
        'unsupervised': ['U'],
        'rule': ['rules']
    }
    """

    def __init__(self,
                 data_dir_path, 
                 file_names_dict, 
                 obj,
                 c1=2.5, c2=2.5):
        self.data_dir_path = data_dir_path
        self.file_names_dict = file_names_dict

        self.c1 = c1
        self.c2 = c2

        # データ
        self.L = None
        self.U = None
        self.S = None 

        # 仮
        self.KB_origin = None
        self.KB = None

        self.KB_tmp = None

        self.predicates_dict_tmp = None
        self.predicates_dict = None

        # ループ用
        self.len_j = None
        self.len_l = None
        self.len_h = None

        self.len_s = None

        self.len_u = None
        self.len_i = None

        # cvxpy.Variable
        self.w_j = None

        self.xi_jl = None
        self.xi_h = None

        self.mu_jl = None
        self.mu_h = None

        self.lambda_jl = None
        self.lambda_hi = None

        self.eta_js = None
        self.eta_hat_js = None


        # coefficients of affine functions
        self.M = None 
        self.q = None

        # evaluation of p for all possible groundings
        self.p_bar = None

        # obj func
        self.obj = obj
        # self.objective_function = None


    @timer
    def load_data(self):
        """
        .csv ファイルからデータを読み込んで，
        辞書を用いて，predicate 名でラベル付けをした
        ndarray として格納する

        {
        'p1': np.array(),
        'p2': np.array(),
        ...
        'pm': np.array()
        }
        """

        # 教師ありデータ
        self.L = {}
        for file_name in self.file_names_dict['supervised']:
            path = os.path.join(self.data_dir_path, file_name)
            self.L[file_name[2:-4]] = np.array(pd.read_csv(path, index_col=0))

        # self.U = {}
        # for file_name in self.file_names_dict['unsupervised']:
        #     path = os.path.join(self.data_dir_path, file_name)
        #     self.U[file_name[2:-4]] = np.array(pd.read_csv(path, index_col=0))

        # 教師なしデータ
        self.U = {}
        for file_name in self.file_names_dict['supervised']:
            path = os.path.join(self.data_dir_path, 'U.csv')
            self.U[file_name[2:-4]] = np.array(pd.read_csv(path, index_col=0))

        # Consistency constraints 用に上の教師ありデータと
        # 教師なしデータを合わせたもの
        self.S = {}
        for key in self.L.keys():
            self.S[key] = np.concatenate((self.L[key][:, :-1] ,self.U[key]), axis=0)


        self.len_j = len(self.L)

        # 仮
        L_tmp = next(iter(self.L.values()))
        self.len_l = len(L_tmp)
        self.dim_x_L = len(L_tmp[0, :-1]) + 1

        U_tmp = next(iter(self.U.values()))
        self.len_u = len(U_tmp)

        self.len_i = 2 * self.len_u

        S_tmp = next(iter(self.S.values()))
        self.len_s = len(S_tmp)

    @timer
    def load_rules(self):
        rule_path = os.path.join(self.data_dir_path, self.file_names_dict['rule'][0])
        fol_processor = FOLConverter(self, rule_path)
        self.KB_info, self.KB_origin, self.KB, self.M, self.q, self.predicates_dict = fol_processor.main()
        
        self.len_h = len(self.KB)

    def _define_cvxpy_variables(self):
        self.w_j = cp.Variable(shape=(self.len_j, self.dim_x_L))

        self.xi_jl = cp.Variable(shape=(self.len_j, self.len_l), nonneg=True)
        self.xi_h = cp.Variable(shape=(self.len_h, 1), nonneg=True)
        
        self.mu_jl = cp.Variable(shape=(self.len_j, self.len_l), nonneg=True)
        self.mu_h = cp.Variable(shape=(self.len_h, 1), nonneg=True)

        self.lambda_jl = cp.Variable(shape=(self.len_j, self.len_l), nonneg=True)
        self.lambda_hi = cp.Variable(shape=(self.len_h, self.len_i), nonneg=True)

        self.eta_js = cp.Variable(shape=(self.len_j, self.len_s), nonneg=True)
        self.eta_hat_js = cp.Variable(shape=(self.len_j, self.len_s), nonneg=True)

    @timer
    def formulate_predicates_with_cvxpy(self):
        """
        KB の中の全 predicate を取得して，辞書に格納．
        predicate function を作成して対応させる
        """
        predicates = self.predicates_dict.keys()
        self._define_cvxpy_variables()
        self.predicates_dict = {predicate: Predicate(self.w_j[j]) for j, predicate in enumerate(predicates)}
    
    @timer
    def construct_constraints(self):
        constraints = []

        for j in range(self.len_j):
            constraint_tmp = 0
            predicate_name = list(self.predicates_dict.keys())[j]
            for h in range(self.len_h):
                for i in range(self.len_i):
                    for u in range(self.len_u):
                        lmbda = self.lambda_hi[h ,i]
                        M = self.M[h][i, u]
                        constraint_tmp += lmbda * M

            for l in range(self.len_l):
                lmbda = self.lambda_jl[j, l]
                y = self.L[predicate_name][l, -1]
                constraint_tmp += -2 * lmbda * y

            for s in range(self.len_s):
                eta = self.eta_js[j, s]
                eta_hat = self.eta_hat_js[j, s]
                constraint_tmp += -1 * (eta - eta_hat)
            
            constraints += [
                constraint_tmp == 0
            ]
        
        for j in range(self.len_j):
            for l in range(self.len_l):
                lmbda = self.lambda_jl[j, l]
                constraints += [
                    lmbda >= 0,
                    lmbda <= self.c1
                ]
        
        for h in range(self.len_h):
            for i in range(self.len_i):
                lmbda = self.lambda_hi[h, i]
                constraints += [
                    lmbda >= 0,
                    lmbda <= self.c2
                ]

        for j in range(self.len_j):
            for s in range(self.len_s):
                eta = self.eta_js[j, s]
                eta_hat = self.eta_hat_js[j, s]
                constraints += [
                    eta >= 0,
                    eta_hat >= 0
                ]

        return constraints
    
    def main(self):
        self.load_data()
        self.load_rules()
        self.formulate_predicates_with_cvxpy()
        objective_function = self.obj(self).construct()
        constraints = self.construct_constraints()
        return objective_function, constraints
        


In [81]:
class ObjectiveFunction:
    def __init__(self, obj: Setup_, kernel_function: object = None) -> None:
        self.L = obj.L
        self.U = obj.U
        self.S = obj.S

        self.len_j = obj.len_j
        self.len_l = obj.len_l
        self.len_u = obj.len_u
        self.len_s = obj.len_s
        self.len_h = obj.len_h 
        self.len_i = obj.len_i

        self.lambda_jl  = obj.lambda_jl
        self.lambda_hi  = obj.lambda_hi
        self.eta_js     = obj.eta_js
        self.eta_hat_js = obj.eta_hat_js

        self.M = obj.M 
        self.q = obj.q

        self.predicate_names = list(obj.predicates_dict.keys())

        if kernel_function == None:
            self.k = self.linear_kernel
        else:
            self.k = kernel_function

    def linear_kernel(self, x1: np.ndarray, x2: np.ndarray) -> float:
        return np.dot(x1, x2)
    
    def _mapping_variables(self) -> Tuple[dict, List[cp.Variable]]:
        mapping_x_i = {}
        x = []

        mapping_x_i["lambda_jl"] = {}
        for j in range(self.len_j):
            for l in range(self.len_l):
                mapping_x_i['lambda_jl'][(j, l)] = len(x)
                x.append(self.lambda_jl[j, l])

        mapping_x_i["lambda_hi"] = {}
        for h in range(self.len_h):
            for i in range(self.len_i):
                mapping_x_i["lambda_hi"][(h, i)] = len(x)
                x.append(self.lambda_hi[h, i])

        mapping_x_i['delta_eta_js'] = {}
        for j in range(self.len_j):
            for s in range(self.len_s):
                mapping_x_i["delta_eta_js"][(j, s)] = len(x)
                x.append(self.eta_js[j, s] - self.eta_hat_js[j, s])
            
        return mapping_x_i, x 
    
    def _construct_P(self) -> Tuple[cp.Variable, np.ndarray]: 
        mapping_x_i, x = self._mapping_variables()
        P = np.zeros((len(x), len(x)))

        for j in range(self.len_j):
            key = self.predicate_names[j]
            L = self.L[key]
            U = self.U[key]
            S = self.S[key]

            for l_row in range(self.len_l):
                row = mapping_x_i['lambda_jl'][(j, l_row)]
                x_row = L[l_row, :-1]
                y_row = L[l_row, -1]
                for l_col in range(self.len_l):
                    col = mapping_x_i['lambda_jl'][(j, l_col)]
                    x_col = L[l_col, :-1]
                    y_col = L[l_col, -1]
                    P[row, col] += 4 * y_row * y_col * self.k(x_row, x_col)
            
            for h_row in range(self.len_h):
                for h_col in range(self.len_h):
                    for i_row in range(self.len_i):
                        for i_col in range(self.len_i):
                            for u_row in range(self.len_u):
                                for u_col in range(self.len_u):
                                    M_row = self.M[h_row][i_row, u_row]
                                    M_col = self.M[h_col][i_col, u_col]
                                    if M_row != 0 and M_col != 0:
                                        row = mapping_x_i['lambda_hi'][(h_row, i_row)]
                                        col = mapping_x_i['lambda_hi'][(h_col, i_col)]
                                        x_row = U[u_row]
                                        x_col = U[u_col]
                                        P[row, col] += M_row * M_col * self.k(x_row, x_col)

            for s_row in range(self.len_s):
                row = mapping_x_i["delta_eta_js"][j, s_row]
                x_row = S[s_row]
                for s_col in range(self.len_s):
                    col = mapping_x_i["delta_eta_js"][j, s_col]
                    x_col = S[s_col]
                    P[row, col] += self.k(x_row, x_col)
                
            for l in range(self.len_l):
                row = mapping_x_i["lambda_jl"][(j, l)]
                x_l = L[l, :-1]
                y_l = L[l, -1]
                for h in range(self.len_h):
                    for i in range(self.len_i):
                        col = mapping_x_i["lambda_hi"][(h, i)]
                        for u in range(self.len_u):
                            x_u = U[u]
                            M = self.M[h][i, u]
                            if M != 0:
                                P[row, col] += -4 * y_l * M * self.k(x_l, x_u)
            
            for l in range(self.len_l):
                row = mapping_x_i["lambda_jl"][(j, l)]
                x_l = L[l, :-1]
                y_l = L[l, -1]
                for s in range(self.len_s):
                    col = mapping_x_i["delta_eta_js"][(j, s)]
                    x_s = S[s]
                    P[row, col] += 4 * y_l * self.k(x_l, x_s)
                
            for h in range(self.len_h):
                for i in range(self.len_i):
                    row = mapping_x_i["lambda_hi"][(h, i)]
                    for s in range(self.len_s):
                        col = mapping_x_i["delta_eta_js"][(j, s)]
                        x_s = S[s]
                        for u in range(self.len_u):
                            x_u = U[u]
                            M = self.M[h][i, u]
                            if M != 0:
                                P[row, col] += -2 * M * self.k(x_u, x_s)
        
        P= (-1/2)*(P+P.T)/2
        return cp.vstack(x), P
    
    # def _construct_q_(self, mapping_x_i: Dict[str, dict]) -> np.ndarray:
    #     q_ = np.zeros((len(x), 1))
    #     col = 0

    #     for j in range(self.len_j):
    #         for l in range(self.len_l):
    #             row = mapping_x_i["lambda_jl"][(j, l)]
    #             q_[row, col] += 1

    #             print(row)
        
    #     for h in range(self.len_h):
    #         for i in range(self.len_i):
    #             row = mapping_x_i["lambda_hi"][(h, i)]
    #             for u in range(self.len_u):
    #                 M = self.M[h][i, u]
    #                 q = self.q[h][i, 0]
    #                 q_[row, col] += (1/2) * M + q

    #             print(row)
        
    #     for j in range(self.len_j):
    #         for s in range(self.len_s):
                 

    def construct(self) -> cp.Expression:
        x, P = self._construct_P()
        objective_function = cp.quad_form(x, P)

        for j in range(self.len_j):
            for l in range(self.len_l):
                objective_function += self.lambda_jl[j, l]
        
        for h in range(self.len_h):
            for i in range(self.len_i):
                for u in range(self.len_u):
                    objective_function += self.lambda_hi[h, i] * (1/2 * self.M[h][i, u] + self.q[h][i, 0])
            
        for j in range(self.len_j):
            for s in range(self.len_s):
                objective_function += (-1/2) * (self.eta_js[j, s] + self.eta_hat_js[j, s])

        objective_function = cp.Maximize(objective_function)

        return objective_function
    

In [122]:
class Predicate_dual:
    def __init__(self, obj: Setup_, name: str, kernel_function: object = None) -> None:
        self.c1 = obj.c1
        self.c2 = obj.c2

        self.L = obj.L[name]
        self.U = obj.U[name]
        self.S = obj.S[name]

        self.len_l = obj.len_l
        self.len_u = obj.len_u
        self.len_s = obj.len_s
        self.len_h = obj.len_h 
        self.len_i = obj.len_i

        self.p_idx = list(obj.predicates_dict.keys()).index(name)

        self.lambda_jl  = obj.lambda_jl[self.p_idx, :].value
        self.lambda_hi  = obj.lambda_hi.value
        self.eta_js     = obj.eta_js[self.p_idx, :].value
        self.eta_hat_js = obj.eta_hat_js[self.p_idx, :].value

        start_col = self.p_idx * self.len_u
        end_col = start_col + self.len_u
        self.M = [M_h[:, start_col:end_col] for M_h in obj.M]

        if kernel_function == None:
            self.k = self.linear_kernel
        else:
            self.k = kernel_function

        self.b = self._b()

    def linear_kernel(self, x1: np.ndarray, x2: np.ndarray) -> float:
        return np.dot(x1, x2)
    
    def w_dot_phi(self, x_pred: np.ndarray) -> float:
        k = self.k
        value = 0

        for l in range(self.len_l):
            x = self.L[l, :-1]
            y = self.L[l, -1]
            lmbda = self.lambda_jl[l]

            value += 2 * lmbda * y * k(x, x_pred)

        for h in range(self.len_h):
            for i in range(self.len_i):
                lmbda = self.lambda_hi[h, i]
                
                for u in range(self.len_u):
                    x = self.U[u]
                    M = self.M[h][i, u]

                    value += - lmbda * M * k(x, x_pred)
        
        for s in range(self.len_s):
            x = self.S[s]
            eta = self.eta_js[s]
            eta_hat = self.eta_hat_js[s]

            value += (eta - eta_hat) * k(x, x_pred)
        
        return value
    
    def _b(self) -> float:
        value_tmp = 0
        count = 0

        for l in range(self.len_l):
            lmbda = self.lambda_jl[l]

            if lmbda >= 0 and lmbda <= self.c1:
                x = self.L[l, :-1]
                y = self.L[l, -1] 

                value_tmp += y - self.w_dot_phi(x)
                count += 1
            
        value = value_tmp / count

        return value

    def __call__(self, x_pred: np.ndarray) -> float:
        
        value = self.w_dot_phi(x_pred) + self.b

        return value


In [136]:
problem_instance = Setup(data_dir_path, file_names_dict, ObjectiveFunction)
objective, constraints = problem_instance.main()

load_data took 0.013332128524780273 seconds!
load_rules took 0.001239776611328125 seconds!
formulate_predicates_with_cvxpy took 0.0003380775451660156 seconds!
construct_constraints took 0.04304790496826172 seconds!


In [137]:
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 22 11:41:15 PM: Your problem has 96 variables, 135 constraints, and 0 parameters.
(CVXPY) Dec 22 11:41:15 PM: It is compliant with the following grammars: DCP, DQCP


(CVXPY) Dec 22 11:41:15 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 22 11:41:15 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 22 11:41:15 PM: Compiling problem (target solver=OSQP).
(CVXPY) Dec 22 11:41:15 PM: Reduction chain: FlipObjective -> CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> OSQP
(CVXPY) Dec 22 11:41:15 PM: Applying reduction FlipObjective
(CVXPY) Dec 22 11:41:15 PM: Applying reduction CvxAttr2Constr
(CVXPY) Dec 22 11:41:15 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Dec 22 11:41:15 PM: Applying reduction QpMatrixStuffing
(CVXPY) Dec 22 11:41:15 PM: Applying reduction OSQP
(CVXPY) D

In [138]:
p1 = Predicate_dual(problem_instance, 'p1')
X = problem_instance.L['p1'][:, :-1]
Y = problem_instance.L['p1'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p1(x))

[0.1 0.5] -1.0 -0.6749766844491083
[0.4 0.4] -1.0 -0.6499872053229678
[0.3 0.8] 1.0 0.07499165368733496
[0.9 0.7] 1.0 0.3249638897720766


In [139]:
p2 = Predicate_dual(problem_instance, 'p2')
X = problem_instance.L['p2'][:, :-1]
Y = problem_instance.L['p2'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p2(x))

[0.1 0.3] -1.0 -4.339954479595144
[0.6 0.4] -1.0 0.9197149621053597
[0.2 0.8] 1.0 0.15678759659904706
[0.7 0.6] 1.0 3.263451920890736


In [140]:
p3 = Predicate_dual(problem_instance, 'p3')
X = problem_instance.L['p3'][:, :-1]
Y = problem_instance.L['p3'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p3(x))

[0.4 0.2] -1.0 -5.174933089224332
[0.9 0.3] -1.0 5.174869703281757
[0.2 0.6] 1.0 -3.374906756147764
[0.5 0.7] 1.0 3.374970142090337


# $\bold x^\top \bold A \bold x$ を作る

In [ ]:
def objective_fucntion_dual(obj: Setup_) -> cp.Expression:
    

In [12]:
# 変数
obj.lambda_jl, obj.lambda_hi, obj.mu_jl, obj.mu_h, obj.eta_js, obj.eta_hat_js

(Variable((3, 4), nonneg=True),
 Variable((2, 12), nonneg=True),
 Variable((3, 4), nonneg=True),
 Variable((2, 1), nonneg=True),
 Variable((3, 10), nonneg=True),
 Variable((3, 10), nonneg=True))

In [ ]:
# データ
obj.L, obj.U, obj.S

In [31]:
# 
obj.M, obj.q

([array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.],
 

In [38]:
# カーネル関数
k = lambda x, y: np.dot(x, y)

In [50]:
# x

mapping_x_i={}
x=[]
mapping_x_i["lambda_jl"]={}
for j in range(obj.len_j):
  for l in range(obj.len_l):
    mapping_x_i["lambda_jl"][(j,l)]=len(x)
    x.append(obj.lambda_jl[j,l])

mapping_x_i["lambda_hi"]={}
for h in range(obj.len_h):
  for i in range(obj.len_I_h):
    mapping_x_i["lambda_hi"][(h,i)]=len(x)
    x.append(obj.lambda_hi[h,i])

mapping_x_i["delta_eta_j_s"]={}
for j in range(obj.len_j):
  for s in range(obj.len_s):
    mapping_x_i["delta_eta_j_s"][(j,s)]=len(x)
    x.append(obj.eta_js[j,s] - obj.eta_hat_js[j,s])

x = cp.vstack(x)

In [55]:
obj.L

{'p1': array([[ 0.1,  0.5, -1. ],
        [ 0.4,  0.4, -1. ],
        [ 0.3,  0.8,  1. ],
        [ 0.9,  0.7,  1. ]]),
 'p2': array([[ 0.1,  0.3, -1. ],
        [ 0.6,  0.4, -1. ],
        [ 0.2,  0.8,  1. ],
        [ 0.7,  0.6,  1. ]]),
 'p3': array([[ 0.4,  0.2, -1. ],
        [ 0.9,  0.3, -1. ],
        [ 0.2,  0.6,  1. ],
        [ 0.5,  0.7,  1. ]])}

In [ ]:
# P


P=np.zeros((len(x),len(x)))

for j in range(len_j):
    for l in range(len_l):
        for l_prime in range(len_l):
            xi=mapping_x_i["lambda_j_l"][(j, l)]
            xj=mapping_x_i["lambda_j_l"][(j, l_prime)]
            P[xi,xj]+=  4 * y_L[j][0, l] * y_L[j][0, l_prime] * k(x_L[j][:, l], x_L[j][:, l_prime])


    # M が 0 のところを条件分岐してみる
    for h in range(len_h):
        for h_prime in range(len_h):
            for i in range(len_I_h):
                for i_prime in range(len_I_h):
                    for u in range(len_u):
                        for u_prime in range(len_u):
                            if M[h][i, u] != 0 and M[h_prime][i_prime, u_prime]!=0:
                                xi=mapping_x_i["lambda_h_i"][(h, i)]
                                xj=mapping_x_i["lambda_h_i"][(h_prime, i_prime)]
                                P[xi,xj] += M[h][i, u] * M[h_prime][i_prime, u_prime] * k(x_U_tmp[:, u], x_U_tmp[:, u_prime])

    for s in range(len_s):
        for s_prime in range(len_s):
            xi=mapping_x_i["delta_eta_j_s"][(j, s)]
            xj=mapping_x_i["delta_eta_j_s"][(j, s_prime)]
            P[xi,xj] +=  k(x_S[j][:, s], x_S[j][:, s_prime])

    for l in range(len_l):
        for h in range(len_h):
            for i in range(len_I_h):
                for u in range(len_u):
                  if M[h][i, u] != 0 :
                      xi=mapping_x_i["lambda_j_l"][(j, l)]
                      xj=mapping_x_i["lambda_h_i"][(h, i)]
                      P[xi,xj] += (-4) * y_L[j][0, l] * M[h][i, u] * k(x_L[j][:, l], x_U_tmp[:, u])

    for l in range(len_l):
        for s in range(len_s):
            xi=mapping_x_i["lambda_j_l"][(j, l)]
            xj=mapping_x_i["delta_eta_j_s"][(j, s)]
            P[xi,xj] += 4 * y_L[j][0, l] * k(x_L[j][:, l], x_S[j][:, s])

    for h in range(len_h):
        for i in range(len_I_h):
            for s in range(len_s):
                for u in range(len_u):
                    xi=mapping_x_i["lambda_h_i"][(h, i)]
                    xj=mapping_x_i["delta_eta_j_s"][(j, s)]

                    P[xi,xj] += (-2) * M[h][i, u] * k(x_U_tmp[:, u], x_S[j][:, s])
                    
P= (-1/2)*(P+P.T)/2

# まずは動作確認

In [34]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cvxpy as cp


L_1 = [(.1, .5, -1), (.4, .4, -1), (.3, .8, 1), (.9, .7, 1)]
L_2 = [(.1, .3, -1), (.6, .4, -1), (.2, .8, 1), (.7, .6, 1)]
L_3 = [(.4, .2, -1), (.9, .3, -1), (.2, .6, 1), (.5, .7, 1)]

U = [(.1, .5), (.3, .7), (.5, .4), (.8, .3), (.9, .2), (1, .5)] # U_1 = U_2 = U_3 = U ということでいいのか


display(L_1)
display(L_2)
display(L_3)
print()
display(U)


U = np.array(U).T

display(U)


x_L_1, y_L_1 = np.array([[x_l[0], x_l[1]] for x_l in L_1]).T, np.array([[y_l[2]] for y_l in L_1]).T
x_L_2, y_L_2 = np.array([[x_l[0], x_l[1]] for x_l in L_2]).T, np.array([[y_l[2]] for y_l in L_2]).T
x_L_3, y_L_3 = np.array([[x_l[0], x_l[1]] for x_l in L_3]).T, np.array([[y_l[2]] for y_l in L_3]).T

display(x_L_1)
display(y_L_1)
print()
display(x_L_2)
display(y_L_2)
print()
display(x_L_3)
display(y_L_3)


x_L = [x_L_1, x_L_2, x_L_3]
x_L


y_L = [y_L_1, y_L_2, y_L_3]
y_L

x_U_1 = U
x_U_2 = U
x_U_3 = U

x_U = [x_U_1, x_U_2, x_U_3]

display(x_U)


x_S_1 = np.hstack([x_U_1, x_L_1])
x_S_2 = np.hstack([x_U_2, x_L_2])
x_S_3 = np.hstack([x_U_3, x_L_3])

x_S = [x_S_1, x_S_2, x_S_3]

x_S

# 6 x 12 の行列
M_1 = np.array([[-1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1]])

M_2 = np.array([[-1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1]])

M = [M_1, M_2]
print(M)


# 6 x 1 の行列（列ベクトル）
q_1 = np.array([[1, 1, 1, 1, 1, 1]]).T
q_2 = np.array([[1, 1, 1, 1, 1, 1]]).T

q = [q_1, q_2]
print(q)


# 通常の内積

def kernel_function(x1, x2):
    return x1.T @ x2

k = kernel_function


# 変数
lambda_j_l = cp.Variable(shape=(3, 4), nonneg=True) # j \in {1, 2, 3}, l \in {1, 2, 3, 4}
lambda_h_i = cp.Variable(shape=(2, 6), nonneg=True) # h \in {1, 2}, i \in {1, 2, 3, 4, 5, 6}
eta_j_s = cp.Variable(shape=(3, 10), nonneg=True) # j \in {1, 2, 3}, s \in {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
eta_hat_j_s = cp.Variable(shape=(3, 10), nonneg=True) # j \in {1, 2, 3}, s \in {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

lambda_j_l, lambda_h_i, eta_j_s, eta_hat_j_s


len_j = 3
len_l = 4

len_h = 2
len_I_h = 6

len_u = 12

len_s = 10

# とりあえずこれで
x_U_tmp = np.hstack([x_U[:2][1], x_U[1:][0]])
x_U_tmp


mapping_x_i={}
# count=0
x=[]
mapping_x_i["lambda_j_l"]={}
for j in range(len_j):
  for l in range(len_l):
    mapping_x_i["lambda_j_l"][(j,l)]=len(x)
    x.append(lambda_j_l[j,l])

mapping_x_i["lambda_h_i"]={}
for h in range(len_h):
  for i in range(len_I_h):
    mapping_x_i["lambda_h_i"][(h,i)]=len(x)
    x.append(lambda_h_i[h,i])

mapping_x_i["delta_eta_j_s"]={}
for j in range(len_j):
  for s in range(len_s):
    mapping_x_i["delta_eta_j_s"][(j,s)]=len(x)
    x.append(eta_j_s[j,s]-eta_hat_j_s[j,s])

print(mapping_x_i)
mapping_x_i

print("#variable:",len(x))

len_j = 3
len_l = 4

len_h = 2
len_I_h = 6

len_u = 12

len_s = 10

# とりあえずこれで
x_U_tmp = np.hstack([x_U[:2][1], x_U[1:][0]])
x_U_tmp

P=np.zeros((len(x),len(x)))

for j in range(len_j):
    for l in range(len_l):
        for l_prime in range(len_l):
            xi=mapping_x_i["lambda_j_l"][(j, l)]
            xj=mapping_x_i["lambda_j_l"][(j, l_prime)]
            P[xi,xj]+=  4 * y_L[j][0, l] * y_L[j][0, l_prime] * k(x_L[j][:, l], x_L[j][:, l_prime])


    # M が 0 のところを条件分岐してみる
    for h in range(len_h):
        for h_prime in range(len_h):
            for i in range(len_I_h):
                for i_prime in range(len_I_h):
                    for u in range(len_u):
                        for u_prime in range(len_u):
                            if M[h][i, u] != 0 and M[h_prime][i_prime, u_prime]!=0:
                                xi=mapping_x_i["lambda_h_i"][(h, i)]
                                xj=mapping_x_i["lambda_h_i"][(h_prime, i_prime)]
                                P[xi,xj] += M[h][i, u] * M[h_prime][i_prime, u_prime] * k(x_U_tmp[:, u], x_U_tmp[:, u_prime])

    for s in range(len_s):
        for s_prime in range(len_s):
            xi=mapping_x_i["delta_eta_j_s"][(j, s)]
            xj=mapping_x_i["delta_eta_j_s"][(j, s_prime)]
            P[xi,xj] +=  k(x_S[j][:, s], x_S[j][:, s_prime])

    for l in range(len_l):
        for h in range(len_h):
            for i in range(len_I_h):
                for u in range(len_u):
                  if M[h][i, u] != 0 :
                      xi=mapping_x_i["lambda_j_l"][(j, l)]
                      xj=mapping_x_i["lambda_h_i"][(h, i)]
                      P[xi,xj] += (-4) * y_L[j][0, l] * M[h][i, u] * k(x_L[j][:, l], x_U_tmp[:, u])

    for l in range(len_l):
        for s in range(len_s):
            xi=mapping_x_i["lambda_j_l"][(j, l)]
            xj=mapping_x_i["delta_eta_j_s"][(j, s)]
            P[xi,xj] += 4 * y_L[j][0, l] * k(x_L[j][:, l], x_S[j][:, s])

    for h in range(len_h):
        for i in range(len_I_h):
            for s in range(len_s):
                for u in range(len_u):
                    xi=mapping_x_i["lambda_h_i"][(h, i)]
                    xj=mapping_x_i["delta_eta_j_s"][(j, s)]

                    P[xi,xj] += (-2) * M[h][i, u] * k(x_U_tmp[:, u], x_S[j][:, s])
P= (-1/2)*(P+P.T)/2


obj_func=cp.quad_form(cp.vstack(x),P)
obj_func


for j in range(len_j):
    for l in range(len_l):
        obj_func += lambda_j_l[j, l]

for h in range(len_h):
    for i in range(len_I_h):
        for u in range(len_u):
            obj_func += lambda_h_i[h, i] * (1/2 * M[h][i, u] + q[h][i, 0])

for j in range(len_j):
    for s in range(len_s):
        obj_func += (-1/2) * (eta_j_s[j, s] - eta_hat_j_s[j, s])

obj_func


# constraint_1 = 0

# for j in range(len_j):
#     for h in range(len_h):
#         for i in range(len_I_h):
#             for u in range(len_u):
#                 constraint_1 += lambda_h_i[h, i] * M[h][i, u]

#     for l in range(len_l):
#         constraint_1 += (-2) * lambda_j_l[j, l] * y_L[j][0, l]

#     for s in range(len_s):
#         constraint_1 += (-1) * (eta_j_s[j, s] - eta_hat_j_s[j, s])


# C_1, C_2 = 2.5, 2.5

# objective = cp.Maximize(obj_func)

# constraints = [
#     constraint_1 == 0,
#     lambda_j_l >=0,
#     lambda_j_l <= C_1,
#     lambda_h_i >= 0,
#     lambda_h_i <= C_2,
#     eta_j_s >=0,
#     eta_hat_j_s >= 0
# ]


# problem = cp.Problem(objective, constraints)
# result = problem.solve(verbose=True)

[(0.1, 0.5, -1), (0.4, 0.4, -1), (0.3, 0.8, 1), (0.9, 0.7, 1)]

[(0.1, 0.3, -1), (0.6, 0.4, -1), (0.2, 0.8, 1), (0.7, 0.6, 1)]

[(0.4, 0.2, -1), (0.9, 0.3, -1), (0.2, 0.6, 1), (0.5, 0.7, 1)]

[(0.1, 0.5), (0.3, 0.7), (0.5, 0.4), (0.8, 0.3), (0.9, 0.2), (1, 0.5)]

array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
       [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])

...

array([[0.4, 0.9, 0.2, 0.5],
       [0.2, 0.3, 0.6, 0.7]])

array([[-1, -1,  1,  1]])

[array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])]

[array([[-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1]]), array([[-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1]])]
[array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]]), array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]])]
{'lambda_j_l': {(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (1, 0): 4, (1, 1): 5, (1, 2): 6, (1, 3): 7, (2, 0): 8, (2, 1): 9, (2, 2): 10, (2, 3): 11}, 'lambda_h_i

{'lambda_j_l': {(0, 0): 0,
  (0, 1): 1,
  (0, 2): 2,
  (0, 3): 3,
  (1, 0): 4,
  (1, 1): 5,
  (1, 2): 6,
  (1, 3): 7,
  (2, 0): 8,
  (2, 1): 9,
  (2, 2): 10,
  (2, 3): 11},
 'lambda_h_i': {(0, 0): 12,
  (0, 1): 13,
  (0, 2): 14,
  (0, 3): 15,
  (0, 4): 16,
  (0, 5): 17,
  (1, 0): 18,
  (1, 1): 19,
  (1, 2): 20,
  (1, 3): 21,
  (1, 4): 22,
  (1, 5): 23},
 'delta_eta_j_s': {(0, 0): 24,
  (0, 1): 25,
  (0, 2): 26,
  (0, 3): 27,
  (0, 4): 28,
  (0, 5): 29,
  (0, 6): 30,
  (0, 7): 31,
  (0, 8): 32,
  (0, 9): 33,
  (1, 0): 34,
  (1, 1): 35,
  (1, 2): 36,
  (1, 3): 37,
  (1, 4): 38,
  (1, 5): 39,
  (1, 6): 40,
  (1, 7): 41,
  (1, 8): 42,
  (1, 9): 43,
  (2, 0): 44,
  (2, 1): 45,
  (2, 2): 46,
  (2, 3): 47,
  (2, 4): 48,
  (2, 5): 49,
  (2, 6): 50,
  (2, 7): 51,
  (2, 8): 52,
  (2, 9): 53}}

In [35]:
x

[Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, NONNEGATIVE, ()),
 Expression(AFFINE, UNKNOWN, ()),
 Expression(AFFINE, UNKNOWN, ()),
 Expression(AFFINE, 

In [59]:
lambda_j_l.value

array([[3.28307247e-01, 2.00439411e-01, 2.50000090e+00, 2.50000084e+00],
       [4.02589365e-05, 3.87174153e-01, 2.50000052e+00, 2.50000014e+00],
       [2.49425874e-05, 2.92620709e-02, 2.49999974e+00, 2.50000114e+00]])

$$\omega^* = 2 \sum_{l=1}^{L}{\lambda_{l}^* y_{l} \phi(\mathbf x_{l})} - \sum_{h=1}^{H}{\sum_{i=1}^{I}{\lambda_{h_i}^*} \sum_{u}^{U}{M^{h}_{i, u} \phi(\mathbf x_u)}} + \sum_{s=1}^{S}{(\eta_s^* - \bar{\eta}_s^*) \phi(\mathbf x_s)}$$

$$\omega^* \cdot \phi(\mathbf x) = 2 \sum_{l=1}^{L}{\lambda_{l}^* y_{l} \phi(\mathbf x_{l}) \cdot \phi(\mathbf x)} - \sum_{h=1}^{H}{\sum_{i=1}^{I}{\lambda_{h_i}^*} \sum_{u}^{U}{M^{h}_{i, u} \phi(\mathbf x_u) \cdot \phi(\mathbf x)}} + \sum_{s=1}^{S}{(\eta_s^* - \bar{\eta}_s^*) \phi(\mathbf x_s) \cdot \phi(\mathbf x)}$$

$$\omega^* \cdot \phi(\mathbf x) = 2 \sum_{l=1}^{L}{\lambda_{l}^* y_{l} k(\mathbf x_{l}, \mathbf x)} - \sum_{h=1}^{H}{\sum_{i=1}^{I}{\lambda_{h_i}^*} \sum_{u}^{U}{M^{h}_{i, u} k(\mathbf x_u, \mathbf x)}} + \sum_{s=1}^{S}{(\eta_s^* - \bar{\eta}_s^*) k(\mathbf x_s, \mathbf x)}$$

$$b_j^* = \frac{1}{\text{number of } \tilde l}  \sum_{\tilde l}(y_{\tilde l} - \omega^*_j \cdot \phi(\mathbf x_{\tilde l}))$$

In [40]:
obj.L

{'p1': array([[ 0.1,  0.5, -1. ],
        [ 0.4,  0.4, -1. ],
        [ 0.3,  0.8,  1. ],
        [ 0.9,  0.7,  1. ]]),
 'p2': array([[ 0.1,  0.3, -1. ],
        [ 0.6,  0.4, -1. ],
        [ 0.2,  0.8,  1. ],
        [ 0.7,  0.6,  1. ]]),
 'p3': array([[ 0.4,  0.2, -1. ],
        [ 0.9,  0.3, -1. ],
        [ 0.2,  0.6,  1. ],
        [ 0.5,  0.7,  1. ]])}

In [45]:
obj.lambda_hi

Variable((2, 12), nonneg=True)

In [46]:
mapping_x_i

{'lambda_j_l': {(0, 0): 0,
  (0, 1): 1,
  (0, 2): 2,
  (0, 3): 3,
  (1, 0): 4,
  (1, 1): 5,
  (1, 2): 6,
  (1, 3): 7,
  (2, 0): 8,
  (2, 1): 9,
  (2, 2): 10,
  (2, 3): 11},
 'lambda_h_i': {(0, 0): 12,
  (0, 1): 13,
  (0, 2): 14,
  (0, 3): 15,
  (0, 4): 16,
  (0, 5): 17,
  (1, 0): 18,
  (1, 1): 19,
  (1, 2): 20,
  (1, 3): 21,
  (1, 4): 22,
  (1, 5): 23},
 'delta_eta_j_s': {(0, 0): 24,
  (0, 1): 25,
  (0, 2): 26,
  (0, 3): 27,
  (0, 4): 28,
  (0, 5): 29,
  (0, 6): 30,
  (0, 7): 31,
  (0, 8): 32,
  (0, 9): 33,
  (1, 0): 34,
  (1, 1): 35,
  (1, 2): 36,
  (1, 3): 37,
  (1, 4): 38,
  (1, 5): 39,
  (1, 6): 40,
  (1, 7): 41,
  (1, 8): 42,
  (1, 9): 43,
  (2, 0): 44,
  (2, 1): 45,
  (2, 2): 46,
  (2, 3): 47,
  (2, 4): 48,
  (2, 5): 49,
  (2, 6): 50,
  (2, 7): 51,
  (2, 8): 52,
  (2, 9): 53}}

In [48]:
q, M

([array([[1],
         [1],
         [1],
         [1],
         [1],
         [1]]),
  array([[1],
         [1],
         [1],
         [1],
         [1],
         [1]])],
 [array([[-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
         [ 0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
         [ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0],
         [ 0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0],
         [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0],
         [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1]]),
  array([[-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
         [ 0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
         [ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0],
         [ 0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0],
         [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0],
         [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1]])])

In [49]:
class ObjectiveFunction:
    def __init__(self, obj: Setup_, kernel_function: object = None) -> None:
        self.L = obj.L
        self.U = obj.U
        self.S = obj.S

        self.len_l = obj.len_l
        self.len_u = obj.len_u
        self.len_s = obj.len_s
        self.len_h = obj.len_h 
        self.len_i = obj.len_I_h # あとで obj.len_i に変更する 

        self.lambda_jl  = obj.lambda_jl
        self.lambda_hi  = obj.lambda_hi
        self.eta_js     = obj.eta_js
        self.eta_hat_js = obj.eta_hat_js

        self.M = obj.M 
        self.q = obj.q

        if kernel_function == None:
            self.k = self.linear_kernel
        else:
            self.k = kernel_function

    def linear_kernel(self, x1: np.ndarray, x2: np.ndarray) -> float:
        return np.dot(x1, x2)
    
    def _mapping_variables(self) -> Tuple[dict, List[cp.Variable]]:
        mapping_x_i = {}
        x = []

        mapping_x_i["lambda_jl"] = {}
        for j in range(self.len_j):
            for l in range(self.len_l):
                mapping_x_i['lambda_jl'][(j, l)] = len(x)
                x.append(self.lambda_jl[j, l])

        mapping_x_i["lambda_hi"] = {}
        for h in range(self.len_h):
            for i in range(self.len_i):
                mapping_x_i["lambda_hi"][(h, i)] = len(x)
                x.append(self.lambda_hi[h, i])

        mapping_x_i['delta_eta_js'] = {}
        for j in range(self.len_j):
            for s in range(self.len_s):
                mapping_x_i["delta_eta_js"][(j, s)] = len(x)
                x.append(self.eta_js[j, s] - self.eta_hat_js[j, s])
            
        return mapping_x_i, x 
    
    def _construct_P(self, mapping_x_i: Dict[str, dict]) -> np.ndarray: 
        P = np.zeros((len(x), len(x)))

        for j in range(self.len_j):
            key = list(self.predicates_dict.keys())[j]
            L = self.L[key]
            U = self.U[key]
            S = self.U[key]

            for l_row in range(self.len_l):
                row = mapping_x_i['lambda_jl'][(j, l_row)]
                x_row = L[l_row, :-1]
                y_row = L[l_row, -1]
                for l_col in range(self.len_l):
                    col = mapping_x_i['lambda_jl'][(j, l_col)]
                    x_col = L[l_col, :-1]
                    y_col = L[l_col, -1]
                    P[row, col] += 4 * y_row * y_col * k(x_row, x_col)
            
            for h_row in range(self.len_h):
                for h_col in range(self.len_h):
                    for i_row in range(self.len_i):
                        for i_col in range(self.len_i):
                            for u_row in range(self.len_u):
                                for u_col in range(self.len_u):
                                    M_row = self.M[h_row][i_row, u_row]
                                    M_col = self.M[h_col][i_col, u_col]
                                    if M_row != 0 and M_col != 0:
                                        row = mapping_x_i['lambda_hi'][(h_row, i_row)]
                                        col = mapping_x_i['lambda_hi'][(h_col, i_col)]
                                        x_row = U[u_row]
                                        x_col = U[u_col]
                                        P[row, col] += M_row * M_col * k(x_row, x_col)

            for s_row in range(self.len_s):
                row = mapping_x_i["delta_eta_js"][j, s_row]
                x_row = S[s_row]
                for s_col in range(self.len_s):
                    col = mapping_x_i["delta_eta_j_s"][j, s_col]
                    x_col = S[s_col]
                    P[row, col] += k(x_row, x_col)
                
            for l in range(self.len_l):
                row = mapping_x_i["lambda_jl"][(j, l)]
                x_l = L[l, :-1]
                y_l = L[l, -1]
                for h in range(self.len_h):
                    for i in range(self.len_i):
                        col = mapping_x_i["lambda_hi"][(h, i)]
                        for u in range(self.len_u):
                            x_u = U[u]
                            M = self.M[h][i, u]
                            if M != 0:
                                P[row, col] += -4 * y_l * M * k(x_l, x_u)
            
            for l in range(self.len_l):
                row = mapping_x_i["lambda_jl"][(j, l)]
                x_l = L[l, :-1]
                y_l = L[l, -1]
                for s in range(self.len_s):
                    col = mapping_x_i["delta_eta_js"][(j, s)]
                    x_s = S[s]
                    P[row, col] += 4 * y_l * k(x_l, x_s)
                
            for h in range(self.len_h):
                for i in range(self.len_i):
                    row = mapping_x_i["lambda_hi"][(h, i)]
                    for s in range(self.len_s):
                        col = mapping_x_i["delta_eta_js"][(j, s)]
                        x_s = S[s]
                        for u in range(self.len_u):
                            x_u = U[u]
                            M = self.M[h][i, u]
                            if M != 0:
                                P[row, col] += -2 * M * k(x_u, x_s)
        
        P= (-1/2)*(P+P.T)/2
        return P
    
    # def _construct_q_(self, mapping_x_i: Dict[str, dict]) -> np.ndarray:
    #     q_ = np.zeros((len(x), 1))
    #     col = 0

    #     for j in range(self.len_j):
    #         for l in range(self.len_l):
    #             row = mapping_x_i["lambda_jl"][(j, l)]
    #             q_[row, col] += 1

    #             print(row)
        
    #     for h in range(self.len_h):
    #         for i in range(self.len_i):
    #             row = mapping_x_i["lambda_hi"][(h, i)]
    #             for u in range(self.len_u):
    #                 M = self.M[h][i, u]
    #                 q = self.q[h][i, 0]
    #                 q_[row, col] += (1/2) * M + q

    #             print(row)
        
    #     for j in range(self.len_j):
    #         for s in range(self.len_s):
                 

    def construct(self) -> cp.Expression:
        mapping_x_i, x = self._mapping_variables()
        P = self._construct_P(mapping_x_i)
        
        objective_function = cp.quad_form(cp.vstack(x), P)

        for j in range(self.len_j):
            for l in range(self.len_l):
                objective_function += self.lambda_jl[j, l]
        
        for h in range(self.len_h):
            for i in range(self.len_i):
                for u in range(self.len_u):
                    objective_function += self.lambda_hi[h, i] * (1/2 * self.M[h][i, u] + self.q[h][i, 0])
            
        for j in range(self.len_j):
            for s in range(self.len_s):
                objective_function += (-1/2) * (self.eta_js[j, s] + self.eta_hat_js[j, s])

        objective_function = cp.Maximize(objective_function)

        return objective_function
    



In [44]:
list(obj.predicates_dict.keys())[0]

'p1'

In [32]:
class Predicate_dual:
    def __init__(self, obj: Setup_, name: str, kernel_function: object = None) -> None:
        self.c1 = obj.c1
        self.c2 = obj.c2

        self.L = obj.L[name]
        self.U = obj.U[name]
        self.S = obj.S[name]

        self.len_l = obj.len_l
        self.len_u = obj.len_u
        self.len_s = obj.len_s
        self.len_h = obj.len_h 
        self.len_i = obj.len_I_h # あとで obj.len_i に変更する 

        self.p_idx = list(obj.predicates_dict.keys()).index(name)

        self.lambda_jl  = obj.lambda_jl[self.p_idx, :].value
        self.lambda_hi  = obj.lambda_hi.value
        self.eta_js     = obj.eta_js[self.p_idx, :].value
        self.eta_hat_js = obj.eta_hat_js[self.p_idx, :].value

        self.M = [M_h[:, self.p_idx * self.len_u] for M_h in obj.M]

        if kernel_function == None:
            self.k = self.linear_kernel
        else:
            self.k = kernel_function

        self.b = self._b()

    def linear_kernel(self, x1: np.ndarray, x2: np.ndarray) -> float:
        return np.dot(x1, x2)
    
    def w_dot_phi(self, x_pred: np.ndarray) -> float:
        k = self.k
        value = 0

        for l in range(self.len_l):
            x = self.L[l, :-1]
            y = self.L[l, -1]
            lmbda = self.lambda_jl[l]

            value += 2 * lmbda * y * k(x, x_pred)

        for h in range(self.len_h):
            for i in range(self.len_i):
                lmbda = self.lambda_hi[h, i]
                
                for u in range(self.len_u):
                    x = self.U[u]
                    M = self.M[h][i, u]

                    val += - lmbda * M * k(x, x_pred)
        
        for s in range(self.len_s):
            x = self.S[s]
            eta = self.eta_js[s]
            eta_hat = self.eta_hat_js[s]

            value += (eta - eta_hat) * k(x, x_pred)
        
        return value
    
    def _b(self) -> float:
        value_tmp = 0
        count = 0

        for l in range(self.len_l):
            lmbda = self.lambda_jl[l]

            if lmbda >= 0 and lmbda <= self.c1:
                x = self.L[l, :-1]
                y = self.L[l, -1] 

                value_tmp += y - self._w_dot_phi(x)
                count += 1
            
        value = value_tmp / count

        return value

    def __call__(self, x_pred: np.ndarray) -> float:
        value = self.w_dot_phi(x_pred) + self.b

        return value


In [114]:
tmp = np.array(L_1)
X_test, y_test = tmp[:, :-1], tmp[:, -1]

X_test, y_test

(array([[0.1, 0.5],
        [0.4, 0.4],
        [0.3, 0.8],
        [0.9, 0.7]]),
 array([-1., -1.,  1.,  1.]))

In [112]:
eta_hat_j_s.value

array([[4.91793619e-01, 2.11416767e+00, 4.55163836e-01, 2.63622749e-01,
        1.74741203e-04, 1.88599658e+00, 4.91793619e-01, 3.00252531e-01,
        2.77044371e+00, 3.04363709e+00],
       [9.01523493e-01, 2.57767139e+00, 4.93886584e-01, 3.29386630e-04,
        5.93059559e-05, 1.67630994e+00, 5.90173342e-05, 5.79973794e-01,
        3.24357080e+00, 2.17003440e+00],
       [1.19060845e+00, 2.94127256e+00, 6.17221201e-01, 2.21989133e-03,
        1.11923005e-04, 1.73410968e+00, 3.69399526e-05, 4.38340789e-02,
        2.06593994e+00, 3.06205087e+00]])

In [115]:
for x in X_test:
    print(p1(x))

[0.33870466]
[-0.58937961]
[0.16552724]
[-1.56912889]


In [121]:
0 <= lambda_j_l.value

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [122]:
lambda_j_l.value <= C_1

array([[ True,  True, False, False],
       [ True,  True, False, False],
       [ True,  True,  True, False]])

In [ ]:
array([[ 7.50000000e-01,  2.00000000e+00, -1.07500000e+00],
       [ 3.19767438e-01,  1.93604651e+00, -6.12790698e-01],
       [ 1.09409846e-10,  2.00000000e+00, -4.00000000e-01]])